In [7]:
from collections import namedtuple
import re

In [28]:
Point = namedtuple("Point", ["x", "y"])
Fold  = namedtuple("Fold",  ["dir", "n"])

def readlines(filename):
    lines = []
    with open(filename, "r") as fh:
        for l in fh:
            lines.append(l.strip())
    return lines

def splitlist(xs, sep):
    out = [[]]
    for x in xs:
        if x == sep:
            out.append([])
        else:
            out[-1].append(x)
    return out

class Paper:
    def __init__(self, points, folds):
        self.points = points
        self.folds  = folds
        self.xmax = max([p.x for p in self.points])
        self.ymax = max([p.y for p in self.points])
        
    def print_paper(self):
        for y in range(self.ymax+1):
            for x in range(self.xmax+1):
                p = Point(x, y)
                if p in self.points:
                    print("#", end="")
                else:
                    print(".", end="")
            print()
    
    fold_patt = re.compile(r"fold along (.)=(\d+)")
    
    @classmethod
    def from_file(cls, filename):
        pointlines, foldlines = splitlist(readlines(filename), "")
        
        points = set()
        for p in pointlines:
            x, y = p.split(",")
            points.add(Point(int(x), int(y)))
        #print(points)
        
        folds = []
        for f in foldlines:
            m = Paper.fold_patt.match(f)
            if m:
                folds.append(Fold(m.group(1), int(m.group(2))))
        #print(folds)
        
        return Paper(points, folds)

test = Paper.from_file("test.txt")
test.print_paper()
print(test.folds)

...#..#..#.
....#......
...........
#..........
...#....#.#
...........
...........
...........
...........
...........
.#....#.##.
....#......
......#...#
#..........
#.#........
[Fold(dir='y', n=7), Fold(dir='x', n=5)]


In [36]:
def fold(paper):
    f = paper.folds[0]
    npoints = set()
    if f.dir == "y":
        for p in paper.points:
            if p.y <= f.n:
                npoints.add(p)
            else:
                npoints.add(Point(p.x, f.n-(p.y-f.n)))
    elif f.dir == "x":
        for p in paper.points:
            if p.x <= f.n:
                npoints.add(p)
            else:
                npoints.add(Point(f.n-(p.x-f.n), p.y))
    return Paper(npoints, paper.folds[1:])

def fold_all(paper, debug=False):
    if debug: paper.print_paper()
    i = 0
    while paper.folds:
        paper = fold(paper)
        i += 1
        if debug: 
            print("-- fold {} --".format(i))
            paper.print_paper()
    return paper

print(len(fold(test).points))

17


In [37]:
inp = Paper.from_file("input.txt")
print(len(fold(inp).points))

607


In [38]:
out = fold_all(inp)
out.print_paper()

.##..###..####.#....###..####.####.#...
#..#.#..#....#.#....#..#.#.......#.#...
#....#..#...#..#....#..#.###....#..#...
#....###...#...#....###..#.....#...#...
#..#.#....#....#....#....#....#....#...
.##..#....####.####.#....#....####.####
